<a href="https://colab.research.google.com/github/elangbijak4/Offensive-Defensive-AI-Framework-for-CyberSecurity/blob/main/PoC_MicroAgent_Malware_AI_Demo_for_Academic_Research_Only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Benign PoC: Local Micro-Agent (Colab safe demo)
# - No filesystem writes
# - No subprocess/os.system
# - No network calls (fake_llm_server is local)
# - Simple policy check before exec

import base64
import json
import platform
from textwrap import dedent

###########################
# 1) Encrypted Prompt (Base64 - simulation of obfuscation)
###########################
plain_prompt_template = dedent("""
You are a safe Python assistant.

You receive this JSON describing the environment:
{env_json}

Task:
Write VALID Python code that defines a function called `generated_main()`.
When called, this function must:
 - Print the string "Hello from generated code".
 - Print the Python version using the platform module.

Important:
 - Only output pure Python code.
 - Do NOT include imports other than `platform`.
 - Do NOT use os, sys, subprocess, socket, open, eval, exec, importlib, ctypes.
""").strip()

# simulate obfuscation/encryption
encrypted_prompt = base64.b64encode(plain_prompt_template.encode('utf-8')).decode('utf-8')

###########################
# 2) Local Micro-Agent primitives
###########################
def decrypt_prompt(enc_str: str) -> str:
    """Decode Base64 'encrypted' prompt back to plain text."""
    return base64.b64decode(enc_str.encode('utf-8')).decode('utf-8')

def collect_environment_info() -> dict:
    """Collect only non-sensitive environment info."""
    return {
        "python_version": platform.python_version(),
        "system": platform.system(),
        "release": platform.release(),
        "machine": platform.machine()
    }

def fill_prompt_template(template: str, env: dict) -> str:
    env_json = json.dumps(env, indent=2)
    return template.format(env_json=env_json)

###########################
# 3) Fake LLM Server (deterministic, safe)
###########################
def fake_llm_server(prompt: str) -> str:
    """
    Simulated LLM: returns a harmless Python code string.
    In real experiments, researchers may replace this with an API call
    to a controlled LLM endpoint that enforces safety policies.
    """
    # A deliberately simple and explicit safe code block
    code = dedent("""
    import platform

    def generated_main():
        print("Hello from generated code")
        print("Python version:", platform.python_version())
    """).strip()
    return code

###########################
# 4) Safety Policy Check before execution
###########################
FORBIDDEN_TOKENS = [
    "os.system", "subprocess", "socket", "open(", "eval(", "exec(",
    "import os", "import sys", "ctypes", "importlib", "from os", "from sys",
    "__import__", "shutil", "fork", "popen"
]

def safe_policy_check(code_str: str) -> None:
    """Raise RuntimeError if code contains forbidden tokens."""
    lower = code_str.lower()
    for token in FORBIDDEN_TOKENS:
        if token in lower:
            raise RuntimeError(f"Unsafe token detected in generated code: {token}")

###########################
# 5) Execute in restricted namespace
###########################
def execute_generated_code(code_str):
    """
    Execute code in a restricted namespace.
    We pass an empty globals dict to reduce possible access to outer scope.
    The local namespace receives only what exec defines.
    """
    # Policy check
    safe_policy_check(code_str)

    # Prepare very restricted globals (only builtins allowed)
    restricted_globals = {
        "__builtins__": {
            # allow only a minimal safe subset of builtins
            "True": True, "False": False, "None": None, "print": print,
            "len": len, "range": range, "__import__": __import__
        },
        "platform": platform # Explicitly add the platform module to the restricted globals
    }

    local_ns = {}
    exec(code_str, restricted_globals, local_ns)
    if "generated_main" not in local_ns:
        raise RuntimeError("generated_main not found in generated code")
    # Call the generated function
    local_ns["generated_main"]()

###########################
# 6) Full agent run (simulate)
###########################
def run_local_agent_demo(enc_prompt: str):
    # 1. decrypt
    template = decrypt_prompt(enc_prompt)
    # 2. collect environment
    env = collect_environment_info()
    # 3. fill template
    filled_prompt = fill_prompt_template(template, env)
    # 4. send to fake LLM server
    generated_code = fake_llm_server(filled_prompt)
    # 5. execute (with safe policies)
    execute_generated_code(generated_code)

# Run demo
if __name__ == "__main__":
    run_local_agent_demo(encrypted_prompt)

Hello from generated code
Python version: 3.12.12
